### Part 1: 將 HDR file 的 E 從 8-bit clip 到 6-bit

In [1]:
import numpy as np

In [2]:
def read_hdr_rgbe(path):
    with open(path, "rb") as f:
        while True:
            line = f.readline().decode(errors="ignore")
            if line.strip()=="":
                break

        line=f.readline().decode().strip().split()
        H=int(line[1])
        W=int(line[3])
        print(f"Width: {W}, Height: {H}")

        img=np.zeros((H,W,4),dtype=np.uint8)

        for y in range(H):
            header=f.read(4)
            if header[0]!=2 or header[1]!=2:
                raise ValueError("Not RLE Radiance HDR")

            scan = np.zeros((W,4),dtype=np.uint8)
            for c in range(4):
                x=0
                while x<W:
                    val=ord(f.read(1))
                    if val>128:   # run
                        cnt=val-128
                        b=ord(f.read(1))
                        scan[x:x+cnt,c]=b
                        x+=cnt
                    else:         # literal
                        raw=f.read(val)
                        scan[x:x+val,c]=list(raw)
                        x+=val
            img[y]=scan
    return img,W,H

In [3]:
def clamp_E_4bit_signed(hdr):
    hdr2=hdr.copy().astype(np.int16)

    E = hdr2[:,:,3]                             # 0~255
    Esigned = E - 128                           # → -128 ~ +127
    Eclamp = np.clip(Esigned, -8, 7)            # → 限制到 4-bit signed
    Enew = (Eclamp + 128).astype(np.uint8)      # → 回到 RGBE 格式

    hdr2[:,:,3] = Enew
    return hdr2.astype(np.uint8)

In [4]:
def write_hdr_rgbe(path,img,W,H):
    with open(path,"wb") as f:
        f.write(b"#?RADIANCE\nFORMAT=32-bit_rle_rgbe\n\n")
        f.write(f"-Y {H} +X {W}\n".encode())

        for y in range(H):
            f.write(bytes([2,2,(W>>8)&255,W&255]))
            scan=img[y]

            for c in range(4):
                x=0
                while x<W:
                    # RLE run block
                    run=1
                    while x+run<W and scan[x+run,c]==scan[x,c] and run<127:
                        run+=1

                    if run>=2:
                        f.write(bytes([128+run,scan[x,c]]))
                        x+=run
                    else:
                        start=x
                        x+=1
                        while x<W and scan[x,c]!=scan[x-1,c] and (x-start)<127:
                            x+=1
                        block=scan[start:x,c].tolist()
                        f.write(bytes([len(block)]))
                        f.write(bytes(block))

    print("✔ saved:",path)

In [45]:
hdr,W,H = read_hdr_rgbe("../hdr_file/Desk.hdr")

hdr4 = clamp_E_4bit_signed(hdr)

write_hdr_rgbe("../hdr_clip/Ocean.hdr", hdr4, W,H)

Width: 644, Height: 874
✔ saved: ../hdr_clip/Ocean.hdr


### Part 2: 將轉好的 HDR 檔案 的 RGBE 資訊 (共 8*3 + 4 = 28 bits) 寫入 dat file 讓 testbench 存入 sram 內 
格式為 R(8) G(8) B(8) E(4) 用 hex 保存 

In [5]:
def rgbe_to_28bit_4bitE_safe(rgbe):
    """
    RGBE array of uint8 (R,G,B,E) → 28-bit int
    E 已經 clip 到 4-bit signed
    """
    # np.uint8 轉 int
    R = int(rgbe[0].item())
    G = int(rgbe[1].item())
    B = int(rgbe[2].item())
    E4 = (int(rgbe[3].item()) - 128) & 0xF

    val = (R << 20) | (G << 12) | (B << 4) | E4
    return val

In [6]:
def write_dat_4bytes(hdr4, filename):
    """
    hdr4: H x W x 4 uint8 RGBE，E 已 clip 到 4-bit signed範圍
    filename: 輸出的 .dat 檔
    每個 pixel 4 bytes (R,G,B,E)，直接 hex 寫入，每行一個 pixel
    """
    H, W, _ = hdr4.shape
    with open(filename, "w") as f:
        for y in range(H):
            for x in range(W):
                pixel = hdr4[y, x]

                # 直接用 4 個 f.write (hex)
                f.write("{:02X}".format(int(pixel[0].item())))  # R
                f.write("{:02X}".format(int(pixel[1].item())))  # G
                f.write("{:02X}".format(int(pixel[2].item())))  # B
                f.write("{:02X}".format(int(pixel[3].item())))  # E
                f.write("\n")  # 每行一個 pixel
    print("✔ DAT file saved:", filename)

In [7]:
dat_path = "../dat_file/input_pattern_0.dat"
hdr,W,H = read_hdr_rgbe("../hdr_file/Desk.hdr")
write_dat_4bytes(hdr, dat_path)

Width: 644, Height: 874
✔ DAT file saved: ../dat_file/input_pattern_0.dat


### Part 3: 固定長寬

In [8]:
def crop_hdr(img, crop_w, crop_h, anchor='center', pad=False):
    """
    Crop a HDR image array (H x W x 4, uint8) to (crop_h, crop_w).
    Params:
      img: np.ndarray shape (H, W, 4), dtype uint8
      crop_w, crop_h: desired crop width and height (ints)
      anchor: 'center' | 'topleft' | 'topright' | 'bottomleft' | 'bottomright'
              or a tuple (x_start, y_start) for explicit pixel offset (0-based).
              For tuple, it's (x, y) of top-left corner of crop in source image.
      pad: if True, allow crop larger than source image and pad missing regions with zeros.
           if False and crop is larger than source, raises ValueError.
    Returns:
      cropped image array shape (crop_h, crop_w, 4), dtype uint8
    """
    if img.ndim != 3 or img.shape[2] != 4:
        raise ValueError("img must be H x W x 4 RGBE uint8 array")

    H, W, _ = img.shape
    if isinstance(anchor, tuple):
        x0, y0 = anchor
    else:
        if anchor == 'center':
            x0 = (W - crop_w) // 2
            y0 = (H - crop_h) // 2
        elif anchor == 'topleft':
            x0 = 0; y0 = 0
        elif anchor == 'topright':
            x0 = W - crop_w; y0 = 0
        elif anchor == 'bottomleft':
            x0 = 0; y0 = H - crop_h
        elif anchor == 'bottomright':
            x0 = W - crop_w; y0 = H - crop_h
        else:
            raise ValueError("unsupported anchor: " + str(anchor))

    # If no padding allowed, require crop entirely inside source
    if not pad:
        if x0 < 0 or y0 < 0 or x0 + crop_w > W or y0 + crop_h > H:
            raise ValueError("Crop region is outside source image bounds; set pad=True to allow padding.")

        return img[y0:y0+crop_h, x0:x0+crop_w].copy()

    # With padding: create zero canvas and place overlapped region
    out = np.zeros((crop_h, crop_w, 4), dtype=np.uint8)

    # compute source region coordinates
    src_x0 = max(0, x0)
    src_y0 = max(0, y0)
    src_x1 = min(W, x0 + crop_w)
    src_y1 = min(H, y0 + crop_h)

    # dest coordinates where to place the src region
    dst_x0 = max(0, -x0)
    dst_y0 = max(0, -y0)
    dst_x1 = dst_x0 + (src_x1 - src_x0)
    dst_y1 = dst_y0 + (src_y1 - src_y0)

    if src_x1 > src_x0 and src_y1 > src_y0:
        out[dst_y0:dst_y1, dst_x0:dst_x1] = img[src_y0:src_y1, src_x0:src_x1]

    return out

In [9]:
def read_hdr_rgbe_crop(path, crop_w=None, crop_h=None, anchor='center', pad=False):
    """
    Read HDR RGBE file and optionally crop to (crop_w, crop_h).
    If crop_w or crop_h is None -> return original.
    Returns: (img, W, H) where img has shape (H, W, 4) (or cropped shape).
    """
    img, W, H = read_hdr_rgbe(path)
    if crop_w is None and crop_h is None:
        return img, W, H

    # default to provided dimension; if only one provided, keep the other from source size
    if crop_w is None:
        crop_w = W
    if crop_h is None:
        crop_h = H

    cropped = crop_hdr(img, crop_w, crop_h, anchor=anchor, pad=pad)
    return cropped, crop_w, crop_h

In [10]:
# 讀取並裁切為 640x480（中間裁切）
img_cropped, Wc, Hc = read_hdr_rgbe_crop("../hdr_file/Desk.hdr", crop_w=640, crop_h=480, anchor='center', pad=False)

# 若要先 clamp E 再裁切（示範）
# hdr, W, H = read_hdr_rgbe("../hdr_file/Desk.hdr")
# hdr4 = clamp_E_4bit_signed(hdr)
# hdr4_cropped = crop_hdr(hdr4, 640, 480, anchor='center', pad=False)

# 寫出裁切後的 HDR
write_hdr_rgbe("../hdr_clip/Desk_640x480.hdr", img_cropped, 640, 480)

# 若要產生 .dat（用你現有的 write_dat_4bytes）
write_dat_4bytes(img_cropped, "../dat_file/Desk_640x480.dat")

Width: 644, Height: 874
✔ saved: ../hdr_clip/Desk_640x480.hdr
✔ DAT file saved: ../dat_file/Desk_640x480.dat
